In [1]:
import pandas as pd
from collections import Counter
import math
import pickle
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import pickle
import json

# As células a seguir importantam os itens e seus respectivos clusters
> Selecione um ou outro, ie, mutualmente exclusivos

In [ ]:
# df = pd.read_csv("../data/output/druid_fasttext/f03_items.csv.zip", sep=";") # itens licitação

# with open("../data/output/druid_fasttext/results.pkl", "rb") as f:
#     groups = pickle.load(f)

In [2]:
df = pd.read_csv("../data/output/nota_fiscal_fasttext/f03_items.csv.zip", sep=";") # itens NF

with open("../data/output/nota_fiscal_fasttext/results.pkl", "rb") as f:
    groups = pickle.load(f)

# As cécluas a seguir removem os outliers e "amarram" os itens aos seus respectivos grupos

In [3]:
# Remove os outliers e agrupa todos os itens de uma mesma família, ie, "gasolina_1", "gasolina_2", ..., "gasolina_n" => "gasolina"
inverse = {}
real_cluster = {}

for cluster, items in groups.items():
    if "_" in cluster:
        group, cod = cluster.split("_")
    else:
        cod = "-1"
        
    for item in items:
        if cod == "-1":
            inverse[item] = "outlier"
            real_cluster[item] = cluster
        else:
            inverse[item] = cluster
            real_cluster[item] = cluster

df["group"] = df.index
df.group = df.group.apply(lambda i: inverse.get(i, "outlier"))

# Salva os grupos originais para análise posterior
df["real_cluster"] = df.index
df.real_cluster = df.real_cluster.apply(lambda i: real_cluster.get(i, "outlier"))

# Processamento

Essa seção:
    1. Transforma o itens pré-processados em uma list
    2. Calcula a frequência de tokens em relação ao grupo
    3. Calcula o score de acordo com a posição que ele aparece na descrição
    

In [4]:
def parse(text):
    tokens = eval(text)
    return tokens

# transforma de str -> list
df.original_prep = df.original_prep.apply(lambda l: parse(l))

In [5]:
# lista todos os grupos com exceção do grupo "outlier"
groups = df.group.drop_duplicates().to_list()
groups.remove('outlier')

In [6]:
# Calcula a frequência global
def calc_token_freq(df):
    freq = Counter()
    
    for item_description in df.original_prep:
        for token in item_description:
            freq[token] += 1
    
    N = max(freq.values())
    for f in freq:
        freq[f] /= N
        
    return freq

# Indentifica a posição que um token t aparece na descrição d
def calc_token_pos(df):
    token_pos = dict()
    for k, row in df.iterrows():
        for i, token in enumerate(row.original_prep):
            if token not in token_pos:
                token_pos[token] = []
            token_pos[token].append(i)
            
    return token_pos

In [ ]:
begin = datetime.now()

group_time = []
group_scores = {}

for group in sorted(groups):
    g_begin = datetime.now()
    indexes = []
    
    _df = df.loc[df.group == group] # Apenas os itens do mesmo grupo
    freq = calc_token_freq(_df) # Calcula a frequência de todos os tokens presentes nas descrições do grupo
    
    token_pos = calc_token_pos(_df) # Calcula a posição que cada termo aparece no grupo
            
    median_tokens_pos, sum_tokens_pos, abs_log_sum_tokens_pos = {}, {}, {}
    for token, pos in token_pos.items():
        median_tokens_pos[token] = np.median(2 ** -np.array(pos, dtype=float))
        sum_tokens_pos[token] = np.sum(2 ** -np.array(pos, dtype=float))
        abs_log_sum_tokens_pos[token] = -np.log2(np.sum(2 ** -np.array(pos, dtype=float)))
    
    group_scores[group] = {"freq": freq, "median": median_tokens_pos, "sum": sum_tokens_pos, "abs_log": abs_log_sum_tokens_pos}
    g_end = datetime.now()
    group_time.append((group, (g_end-g_begin).total_seconds()))
end = datetime.now()
print(f"took {(end-begin).total_seconds()}")

In [ ]:
topN_by_group = {}

for group, scores in group_scores.items():
    tk_median, tk_sum, tk_abs_log = [], [], []
    
    for token, score in scores["freq"].items():
        tk_median.append((token, scores["median"][token] * score))
        tk_sum.append((token, scores["sum"][token]))
        tk_abs_log.append((token, abs(scores["abs_log"][token] * score)))
        
    tk_median.sort(key=lambda v: v[1], reverse=True)
    tk_sum.sort(key=lambda v: v[1], reverse=True)
    tk_abs_log.sort(key=lambda v: v[1], reverse=True)
    
    topN_by_group[group] = [[t[0] for t in tk_median], 
                            [t[0] for t in tk_sum], 
                            [t[0] for t in tk_abs_log]]
 

In [12]:
group_descriptions = []

for group, frequencies in topN_by_group.items():
    g = {"group": group}
    for N in [5, 10, 20, 30]:
        g[f"median_{N}"] = " ".join(sorted(frequencies[0][:N]))
        g[f"sum_{N}"] = " ".join(sorted(frequencies[1][:N]))
        g[f"log_{N}"] = " ".join(sorted(frequencies[2][:N]))
        
    group_descriptions.append(g)

In [13]:
temp_df = pd.DataFrame.from_records(data=group_descriptions).sort_values(by="group")
temp_df["group_prefix"] = [p[0] for p in temp_df.group.str.split("_")]

for N in [5, 10, 20, 30]:
    for metric in ["median", "sum", "log"]:    
        group_df = temp_df.groupby(f"{metric}_{N}")
        print(f"{metric}, {N}, {len(group_df)}")
        
temp_df

median, 5, 18107
sum, 5, 18064
log, 5, 18376
median, 10, 18799
sum, 10, 18800
log, 10, 18782
median, 20, 18822
sum, 20, 18821
log, 20, 18819
median, 30, 18823
sum, 30, 18821
log, 30, 18825


,group,median_5,sum_5,log_5,median_10,sum_10,log_10,median_20,sum_20,log_20,median_30,sum_30,log_30,group_prefix
0,a3_0,a3 cor fotopolimerizavel resina tipo,a3 cor fotopolimerizavel resina tipo,a3 cor fotopolimerizavel resina superior,a3 charisma cor foto fotopolimerizavel padrao ...,a3 charisma cor foto fotopolimerizavel padrao ...,a3 charisma cor fotopolimerizavel heraeus igua...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a3
1,a3_1,5 a3 cor resina z,5 a3 cor resina z,anvisa carga inlays resina seringa,100 5 a100 a3 caixa composto cor repor resina z,100 5 a100 a3 caixa composto cor micro resina z,2 a3 anvisa apresentar carga inlays manipulaca...,100 3 300 5 a100 a3 caixa composto cor fotopol...,0 100 3 4 5 a100 a3 caixa composto cor fotopol...,2 a3 anvisa apresentar carga correcao elevado ...,100 3 300 5 a100 a3 caixa composto cor fotopol...,0 100 3 300 4 5 a100 a3 c caixa composto cor f...,2 a3 anvisa apresentar carga correcao elevado ...,a3
2,a3_2,4 5 a3 resina z,4 5 a3 resina z,4 5 a3 resina restaurador,100 4 5 a3 cor foto gr resina restaurador z,100 4 5 a3 cor foto gr resina restaurador z,4 5 6380 a3 c cor g resina restaurador z,100 2143 250 30 4 4507 5 6032 6380 a3 cor esto...,100 2143 250 30 4 4507 5 6032 6380 a3 cor esto...,10 2277 4 4507 5 6380 7254 7255 a100 a3 bisnag...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,a3
3,aas_0,100 30 aas mg protect,100 30 aas mg protect,100 30 aas mg protect,100 30 7385 aas acetil acido caixa cpr mg protect,100 30 7385 aas acetil acido caixa cpr mg protect,100 30 7385 aas com comprimido mg protect rev ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,aas
4,aas_1,100 aas c comprimido mg,100 aas c comprimido mg,100 aas comprimido cpr mg,100 2286 2448 500 aas c comprimido cp inf mg,100 2286 2448 500 aas asetisin c comprimido in...,10 100 20 200 aas c comprimido cp cpr mg,10 100 1947 200 219 2286 2448 500 aas asetisin...,10 100 1947 219 2286 2448 500 5607 aas asetisi...,10 100 1000 1947 20 200 2448 5607 aas blt c ca...,10 100 1000 1947 200 219 2286 2365 2448 30 453...,10 100 1000 1947 200 219 2286 2365 2448 30 453...,10 100 1000 1947 20 200 219 2286 2365 2448 30 ...,aas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18857,zarcao_3,3 anticorrosivo galao laranja zarcao,3 anticorrosivo galao laranja zarcao,3 6 galao laranja zarcao,1 3 6 anticorrosivo embalagem galao laranja li...,1 3 6 anticorrosivo embalagem galao laranja li...,3 6 anticorrosivo fabricacao galao l laranja p...,0 1 2726 3 4 6 600 9 anticorrosivo com embalag...,0 1 2726 3 4 6 600 9 anticorrosivo com embalag...,1041 2726 3 6 600 anticorrosivo cinza contende...,0 1 1041 2726 3 4 6 600 9 900 anticorrosivo ci...,0 1 1041 2726 3 4 6 600 9 900 anticorrosivo ci...,1041 2726 3 4 6 600 9 900 anticorrosivo cinza ...,zarcao
18858,zinco_0,100 50 60 cm zinco,100 50 cm elementar zinco,cm faixa medicao minim zinco,100 14 40 50 60 aluminio cm elementar solucao ...,100 14 40 50 60 aluminio cm elementar solucao ...,cm compativel d3000 espectrofotometro faixa ha...,1 100 14 3 40 50 60 70 8 aluminio c chapa cm c...,1 100 14 3 40 50 60 70 8 aluminio c chapa cm c...,30 50 analise aproximado caixa capacidade cm c...,1 100 14 3 3241 4 40 5 50 60 70 8 90 aluminio ...,1 100 14 3 3241 4 40 5 50 60 70 8 90 aluminio ...,100 30 50 60 90 agua amostra analise aproximad...,zinco
18859,zinco_1,b exame serico soro zinco,b exame serico soro zin

In [14]:
new_groups = {}
for metric in ["median", "sum", "log"]:
    for n in [5, 10, 20, 30]:
        df_group = temp_df.loc[:, ["group", "group_prefix", f"{metric}_{n}"]].groupby([f"{metric}_{n}"])["group"].apply(list).reset_index()
        
        new_groups[f"{metric}_{n}"] = {}
        for i, row in df_group.iterrows():
            nums = sorted([x.split("_")[1] for x in row["group"]], key=lambda v: int(v))
            group_name = row["group"][0].split("_")[0]
            
            regrouped_name = group_name + "_" + "_".join(nums)
            
            new_groups[f"{metric}_{n}"][regrouped_name] = {
                "description": row[f"{metric}_{n}"],
                "groups": [group_name + "_" + n for n in nums]
            }
            
            

In [ ]:
with open("../data/heuristicas_reagrupamento/new_groups_licitacao.json", "wt", encoding="utf-8") as f:
    json.dump(new_groups, f)

In [ ]:
with open("../data/heuristicas_reagrupamento/terms_score_licitacao.json", "wt", encoding="utf-8") as f:
    json.dump(topN_by_group, f)

In [ ]:
sorted(list(new_groups["log_5"].keys()), key=lambda x: -x.count("_"))

In [16]:
temp_df.loc[temp_df.group == "gasolina_0"]

,group,median_5,sum_5,log_5,median_10,sum_10,log_10,median_20,sum_20,log_20,median_30,sum_30,log_30,group_prefix
8923,gasolina_0,420 838 combustivel comum gasolina,420 838 combustivel comum gasolina,420 491 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,420 428 491 838 combustivel comum gasolina,gasolina


In [17]:
temp_df

,group,median_5,sum_5,log_5,median_10,sum_10,log_10,median_20,sum_20,log_20,median_30,sum_30,log_30,group_prefix
0,a3_0,a3 cor fotopolimerizavel resina tipo,a3 cor fotopolimerizavel resina tipo,a3 cor fotopolimerizavel resina superior,a3 charisma cor foto fotopolimerizavel padrao ...,a3 charisma cor foto fotopolimerizavel padrao ...,a3 charisma cor fotopolimerizavel heraeus igua...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a100 a3 cerar charisma com cor foto fotopolime...,a3
1,a3_1,5 a3 cor resina z,5 a3 cor resina z,anvisa carga inlays resina seringa,100 5 a100 a3 caixa composto cor repor resina z,100 5 a100 a3 caixa composto cor micro resina z,2 a3 anvisa apresentar carga inlays manipulaca...,100 3 300 5 a100 a3 caixa composto cor fotopol...,0 100 3 4 5 a100 a3 caixa composto cor fotopol...,2 a3 anvisa apresentar carga correcao elevado ...,100 3 300 5 a100 a3 caixa composto cor fotopol...,0 100 3 300 4 5 a100 a3 c caixa composto cor f...,2 a3 anvisa apresentar carga correcao elevado ...,a3
2,a3_2,4 5 a3 resina z,4 5 a3 resina z,4 5 a3 resina restaurador,100 4 5 a3 cor foto gr resina restaurador z,100 4 5 a3 cor foto gr resina restaurador z,4 5 6380 a3 c cor g resina restaurador z,100 2143 250 30 4 4507 5 6032 6380 a3 cor esto...,100 2143 250 30 4 4507 5 6032 6380 a3 cor esto...,10 2277 4 4507 5 6380 7254 7255 a100 a3 bisnag...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,10 100 2143 2277 250 30 4 4507 5 6032 6380 725...,a3
3,aas_0,100 30 aas mg protect,100 30 aas mg protect,100 30 aas mg protect,100 30 7385 aas acetil acido caixa cpr mg protect,100 30 7385 aas acetil acido caixa cpr mg protect,100 30 7385 aas com comprimido mg protect rev ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,100 30 7385 aas acetil acido c caixa com comp ...,aas
4,aas_1,100 aas c comprimido mg,100 aas c comprimido mg,100 aas comprimido cpr mg,100 2286 2448 500 aas c comprimido cp inf mg,100 2286 2448 500 aas asetisin c comprimido in...,10 100 20 200 aas c comprimido cp cpr mg,10 100 1947 200 219 2286 2448 500 aas asetisin...,10 100 1947 219 2286 2448 500 5607 aas asetisi...,10 100 1000 1947 20 200 2448 5607 aas blt c ca...,10 100 1000 1947 200 219 2286 2365 2448 30 453...,10 100 1000 1947 200 219 2286 2365 2448 30 453...,10 100 1000 1947 20 200 219 2286 2365 2448 30 ...,aas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18857,zarcao_3,3 anticorrosivo galao laranja zarcao,3 anticorrosivo galao laranja zarcao,3 6 galao laranja zarcao,1 3 6 anticorrosivo embalagem galao laranja li...,1 3 6 anticorrosivo embalagem galao laranja li...,3 6 anticorrosivo fabricacao galao l laranja p...,0 1 2726 3 4 6 600 9 anticorrosivo com embalag...,0 1 2726 3 4 6 600 9 anticorrosivo com embalag...,1041 2726 3 6 600 anticorrosivo cinza contende...,0 1 1041 2726 3 4 6 600 9 900 anticorrosivo ci...,0 1 1041 2726 3 4 6 600 9 900 anticorrosivo ci...,1041 2726 3 4 6 600 9 900 anticorrosivo cinza ...,zarcao
18858,zinco_0,100 50 60 cm zinco,100 50 cm elementar zinco,cm faixa medicao minim zinco,100 14 40 50 60 aluminio cm elementar solucao ...,100 14 40 50 60 aluminio cm elementar solucao ...,cm compativel d3000 espectrofotometro faixa ha...,1 100 14 3 40 50 60 70 8 aluminio c chapa cm c...,1 100 14 3 40 50 60 70 8 aluminio c chapa cm c...,30 50 analise aproximado caixa capacidade cm c...,1 100 14 3 3241 4 40 5 50 60 70 8 90 aluminio ...,1 100 14 3 3241 4 40 5 50 60 70 8 90 aluminio ...,100 30 50 60 90 agua amostra analise aproximad...,zinco
18859,zinco_1,b exame serico soro zinco,b exame serico soro zin

In [18]:
df

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id,group,real_cluster
0,"['prestacao', 'servico', 'eletrico', 'veiculo'...",[],['2.41e+03'],[],[],[],[],63.00,hora,PRESTACAO DE SERVICOS ELETRICOS EM VEICULOS LE...,"[prestacao, servico, eletrico, veiculo, leve, ...",2014,91611180,38706974,0,outlier,prestacao_-1
1,"['oleo', 'diesel', 'combustivel']",[],"['1.00e+01', '2.57e+03']",[],[],[],[],2.71,litro,OLEO DIESEL COMBUSTIVEL S10 2567,"[oleo, diesel, combustivel, 10, 2567]",2014,91611231,38706975,1,oleo_39,oleo_39
2,"['graxo', 'brif', 'ngl2']",[],"['9.00e+00', '2.80e+03']",[],[],[],[],30.00,unidade,G 052737Q9 GRAXA LUBRIF NGL2 2797,"[g, q, 9, graxo, brif, ngl2, 2797]",2014,91638555,38706976,2,graxo_11,graxo_11
3,"['oleo', 'dif']",[],"['9.00e+00', '8.50e+01', '1.40e+02', '2.80e+03']",[],[],[],[],25.00,unidade,G 052815Q9 OLEO DIF 85W140 2798,"[g, q, 9, oleo, dif, 85, w, 140, 2798]",2014,91638555,38706977,3,oleo_55,oleo_55
4,['pneu'],[],"['1.30e+01', '4.72e+02']",[],[],[],[],180.00,unidade,PNEU 16570R13 472,"[pneu, r, 13, 472]",2014,91611257,38706978,4,pneu_26,pneu_26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679112,"['servico', 'mecanico', 'motor', 'freio']",[],['7.00e+03'],[],[],[],[],75.00,hora,SERVICOS MECANICOS DE MOTOR E FREIOS 7001,"[servico, mecanico, motor, freio, 7001]",2020,121762479,44934348,3679112,servico_32,servico_32
3679113,"['contratacao', 'software', 'adiministracao', ...",[],['2.45e+03'],[],[],[],[],6300.00,mes,CONTRATACAO DE SOFTWARE DE ADIMINISTRACAO PUBL...,"[contratacao, software, adiministracao, public...",2020,121762480,44934349,3679113,outlier,contratacao_-1
3679114,"['escada', 'extensivo', 'degrau', 'rebitar']",[],"['3.60e+02', '6.00e+02', '8.74e+03']",[],[],[],[],588.00,unidade,ESCADA EXTENSIVA 360 X 600 DEGRAU REBITADO 8740,"[escada, extensivo, 360, x, 600, degrau, rebit...",2020,121762481,44934350,3679114,escada_6,escada_6
3679115,"['oleo', 'diesel', 'combustivel', 'comum']",[],['4.92e+02'],[],[],[],[],3.75,litro,OLEO DIESEL COMBUSTIVEL COMUM 492,"[oleo, diesel, combustivel, comum, 492]",2020,121762482,44934351,3679115,oleo_4,oleo_4
